In [9]:
import pandas as pd 
data = pd.read_csv("tweets-data/tapera_11-06-2024_13-55-01.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016 entries, 0 to 1015
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      1016 non-null   int64 
 1   created_at               1016 non-null   object
 2   favorite_count           1016 non-null   int64 
 3   full_text                1016 non-null   object
 4   id_str                   1016 non-null   int64 
 5   image_url                742 non-null    object
 6   in_reply_to_screen_name  27 non-null     object
 7   lang                     1016 non-null   object
 8   location                 650 non-null    object
 9   quote_count              1016 non-null   int64 
 10  reply_count              1016 non-null   int64 
 11  retweet_count            1016 non-null   int64 
 12  tweet_url                1016 non-null   object
 13  user_id_str              1016 non-null   int64 
 14  username                 1016 non-null  

In [10]:
data.head(10)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1798014509308879153,Tue Jun 04 15:30:39 +0000 2024,40314,Rumah hasil TAPERA selama 20 tahun https://t.c...,1798014509308879153,https://pbs.twimg.com/media/GPPUqwVakAA0Pdx.jpg,NaN,in,Jakarta Capital Region,934,899,9503,https://x.com/tvindonesiawkwk/status/179801450...,1329295844869214208,tvindonesiawkwk
1,1799771305853820984,Sun Jun 09 11:51:32 +0000 2024,19791,UKT naik: ditunda RUU penyiaran: ditunda Taper...,1799771305853820984,NaN,NaN,in,"Michuhol-gu, Republic of Korea",123,99,5619,https://x.com/ardisatriawan/status/17997713058...,170542374,ardisatriawan
2,1795593859600449863,Tue May 28 23:11:51 +0000 2024,16675,GUYS! Keluargaku udah punya rumah dari 10 tah...,1795593859600449863,https://pbs.twimg.com/media/GOs7GO1aYAAHYQn.jpg,NaN,in,Indonesia,1286,1130,1916,https://x.com/tanyakanrl/status/17955938596004...,1331650559518990336,tanyakanrl
3,1797972060389703950,Tue Jun 04 12:41:58 +0000 2024,58,Janji Prabowo Bikin 3 Juta Rumah Pertahun trus...,1797972060389703950,https://pbs.twimg.com/media/GPOt3xxbIAAod0R.jpg,NaN,in,NaN,5,20,35,https://x.com/165Hati/status/1797972060389703950,1752532657639165952,165Hati
4,1797417985759739963,Mon Jun 03 00:00:16 +0000 2024,800,Masih kesal dengan tapera ini https://t.co/c3u...,1797417985759739963,https://pbs.twimg.com/media/GPG2IaMbYAAmLvG.jpg,NaN,in,Indonesia,4,8,120,https://x.com/tanyakanrl/status/17974179857597...,1331650559518990336,tanyakanrl
5,1797822760858845368,Tue Jun 04 02:48:42 +0000 2024,11461,Keren ini. Rohingya ditampung Gaza ditampung R...,1797822760858845368,https://pbs.twimg.com/media/GPMmRg-bgAAjBdk.jpg,NaN,in,Indonesia,428,831,2719,https://x.com/narkosun/status/1797822760858845368,84221579,narkosun
6,1797255604307988898,Sun Jun 02 13:15:02 +0000 2024,9358,Tapera cuma 3% ga akan membuatmu miskin #eh ht...,1797255604307988898,https://pbs.twimg.com/media/GPEico6aEAEIUsg.jpg,NaN,in,"Data Center, Singapore",107,101,3716,https://x.com/ibnux/status/1797255604307988898,20653807,ibnux
7,1799956165809578136,Mon Jun 10 00:06:06 +0000 2024,10850,Keren ni komika bicara soal anak presiden dan ...,1799956165809578136,https://pbs.twimg.com/ext_tw_video_thumb/17999...,NaN,in,"Planeta Rica, Colombia",112,114,3193,https://x.com/5teV3n_Pe9eL/status/179995616580...,224431376,5teV3n_Pe9eL
8,1799329555909628170,Sat Jun 08 06:36:10 +0000 2024,2142,waduh waduhhh waduhh tapera https://t.co/s2YlF...,1799329555909628170,https://pbs.twimg.com/ext_tw_video_thumb/17993...,NaN,in,NaN,8,47,257,https://x.com/kvpall/status/1799329555909628170,2497536456,kvpall
9,1797605538282561721,Mon Jun 03 12:25:32 +0000 2024,2209,Gaji dipotong TAPERA TAbungan PErumahan meRmAi...,1797605538282561721,https://pbs.twimg.com/media/GPJgsUwasAE_6WU.jpg,NaN,in,NaN,64,20,373,https://x.com/tuna_and_mayo_/status/1797605538...,1675860890518978563,tuna_and_mayo_
